# Load environment

In [22]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(override=True)



True

In [4]:
import pickle

def pickleSave(object, name, folder='.', silent=False):

    filename = folder + '/' + name + '.pkl'
    if silent==False:
        print('Saving object {} to pickle file {}'.format(name, filename))
    with open(filename, mode='wb') as fipkl:    
        pickle.dump(object, fipkl)
        
def pickleLoad(name, folder: str, silent=False):
    
    filename = folder + '/' + name + '.pkl'
    if silent==False:
        print('Loading object {} from pickle file {}'.format(name, filename))
    
    try:
        with open(filename, mode='rb') as fipkl:
            myObject=pickle.load(fipkl)
        return myObject
    except IOError:
        print('Pickle file {} not found, returning None object'.format(filename))
        return None

# Additional private data  

!!! This section only need to be run once because it will store embeddings of the scraped webiste in a persistent instance of ChromaDb  

## Installing persistent storage ChromaDB for our embeddings  

https://howaibuildthis.substack.com/p/a-guide-to-installing-chromadb-on  

How to install ChromaDB:  

git clone https://github.com/chroma-core/chroma  

cd chroma  

docker-compose up -d --build  






## Setting up tiktoken 

... for couting the number of tokens in a string of text  

In [5]:
import tiktoken

def concatenate_page_contents(input_documents, separator):
    page_contents = [doc.dict()['page_content'] for doc in input_documents]
    return separator.join(page_contents)

def num_tokens_from_string(string: str, model_name: str = 'gpt-3.5-turbo') -> int:
    encoding = tiktoken.encoding_for_model(model_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

## Loader: Scraping website

In [7]:
# This will run the actor on Apify to scrape your website !!!

from langchain.document_loaders.base import Document
from langchain.utilities import ApifyWrapper

def getScrapingLoader(url):
    loader = ApifyWrapper().call_actor(
        actor_id='apify/website-content-crawler',
        run_input={'startUrls': [{'url': url}]},
        dataset_mapping_function=lambda item: Document(
            page_content=item['text'] or '', 
            metadata={'source': item['url']}
        ),
    )
    return loader

loader = getScrapingLoader(url = 'https://www.steinel.com.au')





## Using loader

This will:  
  
0. setup chroma client_settings (where is my chroma instance, here assuming it runs on a container)  
1. scrape the website into 'documents'  
2. split those 'documents' into chunks of text  
3. create embeddings from those chunks (using default openai embedding text-davinci-003) and store them in Chroma collection LANTERNPAY_COLLECTION

In [7]:
# information about our instance of Chroma

from chromadb.config import Settings
from chromadb import Client

client_settings = Settings(
        chroma_api_impl="rest",
        chroma_server_host="host.docker.internal",  # when you run this inside a devcontainer you need to explicitely say host.docker.internal to signify "devcontainer host localhost"
        chroma_server_http_port="8000"
    )

# about host.docker.internal : https://stackoverflow.com/questions/24319662/from-inside-of-a-docker-container-how-do-i-connect-to-the-localhost-of-the-mach


In [8]:
# loading data "manually" same as doing VectorstoreIndexCreator().from_loaders([loader])

from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

scraped_documents = loader.load()

### Pickling scraped documents

In [15]:
pickleSave(scraped_documents, 'steinel_scraped_documents', folder='.', silent=False)

Saving object steinel_scraped_documents to pickle file ./steinel_scraped_documents.pkl


In [5]:
scraped_documents = pickleLoad(name='steinel_scraped_documents', folder='.', silent=False)

Loading object steinel_scraped_documents from pickle file ./steinel_scraped_documents.pkl


In [6]:
len(scraped_documents)

171

In [22]:
from pprint import pprint
pprint(scraped_documents[10].page_content)
pprint(scraped_documents[10].metadata['source'])
pprint(scraped_documents[11].page_content)
pprint(scraped_documents[11].metadata['source'])
pprint(scraped_documents[111].page_content)
pprint(scraped_documents[111].metadata['source'])
print('Total number of pages scraped:', len(scraped_documents))
for doc in scraped_documents:
    pprint(doc.metadata['source'])
    print('Page content text length:', len(doc.page_content))
    print('Number of tokens:', num_tokens_from_string(doc.page_content))


('The ultrasonic waves are actively emitted from the sensors and completely '
 'fill the space they cover. Highly sensitive and able to detect minor '
 'movements, the ultrasonic sensors are effective for low motion activities, '
 'which is why they are particularly recommended for rooms in which sedentary '
 'activities are performed.\n'
 'Ultrasonic sensors also detect a larger area than a PIR sensor, meaning '
 'fewer sensors would be required. \n'
 'However, they cannot be mounted higher than 4.2 metres with the sensors '
 'effective range declining proportionally to partition height. They are very '
 'susceptible to high levels of vibration of airflow and not adjustable so not '
 'ideal for areas that require selective coverage.')
'https://www.steinel.com.au/product-category/ultrasonic-sensors/'
('The STEINEL indoor sensorlights combine state-of-the-art sensor technology '
 'with sleek design. Integrated sensors are invisibly concealed inside these '
 'lights to instantly respond 

In [11]:

splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=250)
texts = splitter.split_documents(scraped_documents)

In [23]:
for t in texts[1:5]:
    print('metadata:', t.metadata['source'])
    print('text length:', len(t.page_content))
    print('num_token:', num_tokens_from_string(t.page_content))

metadata: https://www.steinel.com.au/latest-news/
text length: 952
num_token: 221
metadata: https://www.steinel.com.au/contact-us/
text length: 134
num_token: 39
metadata: https://www.steinel.com.au/about-us/
text length: 679
num_token: 121
metadata: https://www.steinel.com.au/product/solar-light-xsolar-l2-s-house-number-silver/
text length: 858
num_token: 183


In [24]:
db = Chroma.from_documents(texts, client_settings=client_settings, embedding = OpenAIEmbeddings(), collection_name='STEINEL_COLLECTION')
retriever = db.as_retriever()

### Problem max query size exceeded from Chroma

clickhouse_connect.driver.httpclient Code: 62. DB::Exception: Syntax error: failed at position 262129 ('0.02478806984617376') (line 108, col 14488): 0.02478806984617376, 0.010010567241768666, -0.016834096559357397, 0.015376775138473653, 0.005069977476240352, -0.022758718007165845, -0.05262698100664378, -0.01. Max query size exceeded: '0.02478806984617376'. (SYNTAX_ERROR) (version 22.9.7.34 (official build)

(found in docker logs for Chroma)

Solution: https://github.com/chroma-core/chroma/issues/687  


# Part 2. Once website has been scraped and store in Chroma DB  



In [8]:
# check chroma db works (you always have to provide the embedding function)

from chromadb import Client

from langchain.embeddings import OpenAIEmbeddings
chromaClient = Client(client_settings)
coll = chromaClient.get_collection(name='STEINEL_COLLECTION', embedding_function=OpenAIEmbeddings())


We used the Chroma client to get our collection. Now we can view the collection name and also find out what has been stored 
in the database.



In [8]:
coll.dict()

{'name': 'STEINEL_COLLECTION',
 'id': UUID('9345a1cc-0479-4614-adef-1f2df41f49e9'),
 'metadata': None}

In [9]:
coll.peek()

{'ids': ['d8a094e6-2901-11ee-a2b0-0242ac110002',
  'd8a09afe-2901-11ee-a2b0-0242ac110002',
  'd8a09c16-2901-11ee-a2b0-0242ac110002',
  'd8a09c98-2901-11ee-a2b0-0242ac110002',
  'd8a09cfc-2901-11ee-a2b0-0242ac110002',
  'd8a09d60-2901-11ee-a2b0-0242ac110002',
  'd8a09e5a-2901-11ee-a2b0-0242ac110002',
  'd8a09f0e-2901-11ee-a2b0-0242ac110002',
  'd8a09f86-2901-11ee-a2b0-0242ac110002',
  'd8a09fea-2901-11ee-a2b0-0242ac110002'],
 'embeddings': [[0.0060437328388491685,
   -0.00869814243142026,
   -0.013389502179035232,
   -0.023919892384897667,
   0.004553773130650533,
   0.01500698137248337,
   -0.019705052230800768,
   -0.032027420752923016,
   -0.008839085069397215,
   -0.004040341031285222,
   0.006694751089517616,
   0.011993503608606165,
   -0.006516895480032119,
   -0.005520233607768919,
   -0.015302287938519522,
   -0.003328918826173886,
   0.017127823670936668,
   0.018456707409051077,
   0.03575903299676724,
   -0.016188210430595793,
   -0.014000251437182625,
   0.01834932218723145

# Loading from persistent vectorstore ChromaDB

In [9]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from chromadb.config import Settings

client_settings = Settings(
        chroma_api_impl="rest",
        chroma_server_host="host.docker.internal",  # when you run this inside a devcontainer you need to explicitely say host.docker.internal to signify "devcontainer host localhost"
        chroma_server_http_port="8000"
    )

chromaDb = Chroma(collection_name='STEINEL_COLLECTION', 
                     embedding_function=OpenAIEmbeddings(),
                     client_settings=client_settings)



## Creating a RetrievalQA chain

This type of chain is used to find information using a retriever.  
A retriever is an interface that returns relevant documents given an unstructured query.  
It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) it.  
Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.  

Here our retriever is going to be made using langchain chromaDb.as_retriever(search_kwargs={"k": 1})  



In [10]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model='gpt-3.5-turbo')

retriever = chromaDb.as_retriever(search_kwargs={"k": 4})

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type='stuff',
    retriever=retriever,
    # chain_type_kwargs={"prompt": PROMPT}, 
    # return_source_documents=True,
    verbose=True
)

# note: the search kwargs is to limit what is sent to the llm, i.e. to change the number of document to return from retriever
# retriever.search_kwargs['k'] = 10

In [8]:
from pprint import pprint

query = "What is Steinel product offering?"
# result = qa.run(query)
result = qa({"query": query})

pprint(result)

# InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 10515 tokens. Please reduce the length of the messages.

Inputs: {'query': 'What is Steinel product offering?'}
Count Inputs: 1


> Entering new RetrievalQA chain...
Chroma Similarity search: [(Document(page_content='From developing the world’s first infrared sensor, Steinel has maintained its leadership and ability to commercialise core R&D capability in sensors and sensorlights. Founded in 1959 in Germany, with sales in over 80 countries, Steinel is a global leader in lighting efficiency.\nSteinel boasts an extensive range of intelligent sensors and integrated sensor lights providing solutions across various industries including: Residential, Commercial, Education, Retail, Industrial, Healthcare, Hospitality and Community Sectors.\nWith offices in Sydney and Melbourne, and technical partners all over the Country, we pride ourselves on our ability to assist customers across Australia.', metadata={'source': 'https://www.steinel.com.au/about-us/'}), 0.23675300180912018), (Document(page_content='From developing the world’s first infrared senso

In [9]:
query = "Do you have sensors that detects movements in corridor?"
# result = qa.run(query)
result = qa.run(query)

pprint(result)

Inputs: {'query': 'Do you have sensors that detects movements in corridor?'}
Count Inputs: 1


> Entering new RetrievalQA chain...
Chroma Similarity search: [(Document(page_content='Presence Detector Single US KNX\nThe ideal sensor for small corridors. Single US KNX ultrasonic motion detector, ideal for corridors, thoroughfares and stairwells as well as WC facilities, washrooms and store rooms Detection within 10 m in one direction, both radially and tangentially, envelops objects as opposed to penetrating them, detection regardless of temperature, electronically adjustable reach, with digital BUS interface for controlling 4 lighting channels and HVAC.\nVersion\nAbout Presence Detector Single US KNX\nThe ideal sensor for smaller corridors.\nSimple tasks demand simple answers. This is demonstrated by our Single US 180° ultrasonic sensor. Correctly aligned, it detects any movement in corridors and passageways of up to 10 metres in length – even it they take place behind objects. Movement

In [10]:
query = "Give me 3 motion sensor that you're selling?"
# result = qa.run(query)
documents = retriever.get_relevant_documents(query)
result = qa.combine_documents_chain(inputs={'question': query, 'input_documents': documents})

Chroma Similarity search: [(Document(page_content='Infrared Motion Detector IS 360-3 \nInfrared Motion Detector IS 2360-3 ECO WHITE \nInfrared Motion Detector IS 3360 MX Highbay \nHigh Frequency Motion Detector HF 3360 Square \nHigh Frequency Motion Detector HF 3360 Round \nInfrared Motion Detector IS 3360 Square \nInfrared Motion Detector IS 3360 Round \nInfrared Motion Detector IS 2360 DE ECO White Recessed \nInfrared Motion Detector IS 360-1 DE Black Recessed \nInfrared Motion Detector IS 1 \nInfrared Motion Detector sensIQ S \nInfrared Motion Detector IS 140-2 \nInfrared Motion Detector IS 2180-2 \nInfrared Motion Detector IS 2180-5 \nInfrared Motion Detector IS 2360 \nIntelligent Motion Detector iHF 3D \nInfrared Motion Detector IS D 3360 \nInfrared Motion Detector HBS 300 Highbay \nInfrared Motion Detector IS 345 MX Highbay \nInfrared Motion Detector IS 345 MX Highbay – no remote control \nInfrared Motion Detector IS 3180 \nInfrared Motion Detector IS 300 \nPhotoelectric Lighting

In [11]:
print(documents[0].dict().keys())

dict_keys(['page_content', 'metadata'])


In [12]:
pprint(result)

{'input_documents': [Document(page_content='Infrared Motion Detector IS 360-3 \nInfrared Motion Detector IS 2360-3 ECO WHITE \nInfrared Motion Detector IS 3360 MX Highbay \nHigh Frequency Motion Detector HF 3360 Square \nHigh Frequency Motion Detector HF 3360 Round \nInfrared Motion Detector IS 3360 Square \nInfrared Motion Detector IS 3360 Round \nInfrared Motion Detector IS 2360 DE ECO White Recessed \nInfrared Motion Detector IS 360-1 DE Black Recessed \nInfrared Motion Detector IS 1 \nInfrared Motion Detector sensIQ S \nInfrared Motion Detector IS 140-2 \nInfrared Motion Detector IS 2180-2 \nInfrared Motion Detector IS 2180-5 \nInfrared Motion Detector IS 2360 \nIntelligent Motion Detector iHF 3D \nInfrared Motion Detector IS D 3360 \nInfrared Motion Detector HBS 300 Highbay \nInfrared Motion Detector IS 345 MX Highbay \nInfrared Motion Detector IS 345 MX Highbay – no remote control \nInfrared Motion Detector IS 3180 \nInfrared Motion Detector IS 300 \nPhotoelectric Lighting Contro

In [13]:
# debugging the score of the search

chromaDb.similarity_search_with_score(query)

[(Document(page_content='Infrared Motion Detector IS 360-3 \nInfrared Motion Detector IS 2360-3 ECO WHITE \nInfrared Motion Detector IS 3360 MX Highbay \nHigh Frequency Motion Detector HF 3360 Square \nHigh Frequency Motion Detector HF 3360 Round \nInfrared Motion Detector IS 3360 Square \nInfrared Motion Detector IS 3360 Round \nInfrared Motion Detector IS 2360 DE ECO White Recessed \nInfrared Motion Detector IS 360-1 DE Black Recessed \nInfrared Motion Detector IS 1 \nInfrared Motion Detector sensIQ S \nInfrared Motion Detector IS 140-2 \nInfrared Motion Detector IS 2180-2 \nInfrared Motion Detector IS 2180-5 \nInfrared Motion Detector IS 2360 \nIntelligent Motion Detector iHF 3D \nInfrared Motion Detector IS D 3360 \nInfrared Motion Detector HBS 300 Highbay \nInfrared Motion Detector IS 345 MX Highbay \nInfrared Motion Detector IS 345 MX Highbay – no remote control \nInfrared Motion Detector IS 3180 \nInfrared Motion Detector IS 300 \nPhotoelectric Lighting Controller NightMatic 500

In [11]:
from langchain.agents import Tool

def qaTool(query):
    return qa({"query": query})

name = """steinel_tool"""

description = """Useful for questions about Steinel services and products (sensors and lighting products for residential and commercial use)"""

steinel_tool = Tool(
    name=name,
    func=qa.run,
    description=description,
)


## Make chatbot (agent) with tools


### custom output parser

In [6]:
import re
from typing import Union

from langchain.agents.agent import AgentOutputParser
# from langchain.agents.conversational.prompt import FORMAT_INSTRUCTIONS
from langchain.schema import AgentAction, AgentFinish, OutputParserException

FORMAT_INSTRUCTIONS = """To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, you MUST use the format:

```
Thought: Do I need to use a tool? No
{ai_prefix}: [your response here]
```"""


class ConvoOutputParser2(AgentOutputParser):
    ai_prefix: str = "AI"

    def get_format_instructions(self) -> str:
        return FORMAT_INSTRUCTIONS

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        
        print('***********************************************************')
        print(f'{text}')
        print(f'output AgentFinish:{text.split(f"{self.ai_prefix}:")[-1].strip()}')
        
        if f"{self.ai_prefix}:" in text:
            return AgentFinish(
                {"output": text.split(f"{self.ai_prefix}:")[-1].strip()}, text
            )
        regex = r"Action: (.*?)[\n]*Action Input: (.*)"
        match = re.search(regex, text)
        if not match:
            raise OutputParserException(f"Could not parse LLM output: `{text}`")
        action = match.group(1)
        action_input = match.group(2)
        return AgentAction(action.strip(), action_input.strip(" ").strip('"'), text)

    @property
    def _type(self) -> str:
        return "conversational"

### agent executor with tools

In [14]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent, load_tools, AgentType

llm = ChatOpenAI()

tools = load_tools([
    'wikipedia', 
    'llm-math', 
    'google-search'
], llm=llm)


# tools.append(steinel_tool)

memory = ConversationBufferMemory(memory_key='chat_history')

agent_zero_shot = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, 
    # agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    memory=memory
    # output_parser=ConvoOutputParser2.parse
)

In [15]:
response = agent_zero_shot.run(input="What kind of products does Steinel website sell?")

Inputs: {'input': 'What kind of products does Steinel website sell?', 'chat_history': ''}
Count Inputs: 2


> Entering new AgentExecutor chain...
Inputs: {'input': 'What kind of products does Steinel website sell?', 'chat_history': '', 'agent_scratchpad': '', 'stop': ['\nObservation:', '\n\tObservation:']}
Count Inputs: 4
Thought: Do I need to use a tool? Yes
Action: Wikipedia
Action Input: Steinel website
Observation: Page: Family (2018 film)
Summary: Family is a 2018 American comedy film written and directed by Laura Steinel in her directorial debut. It stars Taylor Schilling, Brian Tyree Henry,  Bryn Vale, Allison Tolman, Fabrizio Guido, Kate McKinnon and Peter Horton. It premiered at South by Southwest on March 11, 2018, and was released on April 19, 2019 by The Film Arcade.
It received positive reviews. Critics praised its comedy and plot twists, and Schilling's performance.



Page: .458 SOCOM
Summary: The .458 SOCOM (11.63×40mm) is a moderately large round designed to work in an

OutputParserException: Could not parse LLM output: `Do I need to use a tool? Yes`

## Random agent 

https://lablab.ai/t/ai-agents-tutorial-how-to-use-and-create-them

In [19]:
import requests

from langchain import OpenAI
from langchain.agents import initialize_agent, load_tools, Tool
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.tools import DuckDuckGoSearchRun

llm = OpenAI(
    temperature=0.8,
    model_name="text-davinci-003"
)

prompt = PromptTemplate(
  input_variables=["query"],
  template="You are New Native Internal Bot. Help users with their important tasks, like a professor in a particular field. Query: {query}"
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

In [20]:
# tools

search = DuckDuckGoSearchRun()

# Web Search Tool
search_tool = Tool(
    name = "Web Search",
    func=search.run,
    description="A useful tool for searching the Internet to find information on world events, issues, etc. Worth using for general topics. Use precise questions."
)

In [24]:
class WA:
  """
    Wolfram|Alpha API
  """
  def __init__(self, app_id):
    self.url = f"http://api.wolframalpha.com/v1/result?appid={app_id}&i="

  def run(self, query):
    query = query.replace("+", " plus ").replace("-", " minus ") # '+' and '-' are used in URI and cannot be used in request
    result = requests.post(f"{self.url}{query}")

    if not result.ok:
      raise Exception("Cannot call WA API.")

    return result.text

In [25]:
# https://products.wolframalpha.com/api/
WA_API_KEY = os.getenv("WA_API_KEY")
wa = WA(app_id=WA_API_KEY)

wa_tool = Tool(
    name="Wolfram|Alpha API",
    func=wa.run,
    description="Wolfram|Alpha API. It's super powerful Math tool. Use it for simple & complex math tasks."
)

In [30]:
agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=[search_tool],
    llm=llm,
    verbose=True, # I will use verbose=True to check process of choosing tool by Agent
    max_iterations=3
)

In [31]:
result = agent('can you list all sensors that steinel.com.au is selling, and which ones are relevant for motion sensing?')

Inputs: {'input': 'can you list all sensors that steinel.com.au is selling, and which ones are relevant for motion sensing?'}
Count Inputs: 1


> Entering new AgentExecutor chain...
Inputs: {'input': 'can you list all sensors that steinel.com.au is selling, and which ones are relevant for motion sensing?', 'agent_scratchpad': '', 'stop': ['\nObservation:', '\n\tObservation:']}
Count Inputs: 3
 I need to find out what sensors are available and which ones are best for motion sensing
Action: Web Search
Action Input: Steinel.com.au sensors for motion sensing
Observation: Steinel Australia Efficient Lighting Technologies. 1. Wide range of sensors & sensor lights. 2. World's most sensitive sensors. 3. 5 years warranty on PRO products. 4. Technical Support available. 5. Motion detector IR 180 AP easy. Motion detector IS 360-1 DE. Motion detector IS 130-2. Motion detector IS 130-2. Motion detector IS 1. Motion detector IS 180 digi. Motion detector IS 360-3. Motion detector IS 140-2. Motion det

In [27]:
r_1 = agent("What is lablab.ai?")
print(f"Final answer: {r_1['output']}")

r_2 = agent("Integral of x * (log(x)^2)")
print(f"Final answer: {r_2['output']}")

Inputs: {'input': 'What is lablab.ai?'}
Count Inputs: 1


> Entering new AgentExecutor chain...
Inputs: {'input': 'What is lablab.ai?', 'agent_scratchpad': '', 'stop': ['\nObservation:', '\n\tObservation:']}
Count Inputs: 3
 I should find out what this is.
Action: Web Search
Action Input: "lablab.ai"
Observation: Plug into AI with AI21. 🗓️ This will be a 10-day virtual hackathon from June 28 to July 7 💻 Access AI21 Labs' state-of-the-art language models to build innovative applications 💡 Meet and learn from AI21 Labs and Lablab experts 👥 Join the community and find your team 🏆 Prizes and awards of up to $9500 API credits + $3500 cash ⭐ Receive a certificate of completion for ... speech and text analysis translations explanations of complex issues writing stories and essays learn coding debugging code OpenAI has not yet released an official API for ChatGPT, but there are several community-created libraries available. Check out our list of ChatGPT API libraries on the OpenAI ChatGPT tech